| ![nsdf](https://www.sci.utah.edu/~pascucci/public/NSDF-smaller.PNG)  | [National Science Data Fabric](https://nationalsciencedatafabric.org/) [Jupyter notebook](https://jupyter.org/) <br> created by  [Valerio Pascucci](http://cedmav.com/) and  [Giorgio Scorzelli](https://www.sci.utah.edu/people/scrgiorgio.html)  |  
|---|:---:|


#  <center> This is a preview of the data:
![Visualization of Covid-19 cases](https://www.sci.utah.edu/~pascucci/public/DRP-preview.gif)

# Import OpenVisus and load dataset

In [ ]:
import os,sys,logging,time


sys.path.append("c:/projects/openvisus/build/RelWithDebInfo")
sys.path.append("c:/projects/openvisuspy/src")

# change as needed
os.environ["VISUS_CPP_VERBOSE"       ]="0"
os.environ["VISUS_NETSERVICE_VERBOSE"]="0"

from openvisuspy import LoadDataset, ExecuteBoxQuery, SetupLogger ,GetBackend
logger=SetupLogger(stream=False)
logger.info(f"GetBackend={GetBackend()}")

url='http://atlantis.sci.utah.edu:80/mod_visus?dataset=fly_scan_time-s-midx/data&cached=1'
db=LoadDataset(url)

print(f"Loaded dataset")
print(f"  fields={db.getFields()}")
print(f"  logic_box={db.getLogicBox()}")
W,H,D=db.getLogicBox()[1]
access=db.createAccess()

# Test panel volume rendererf

In [ ]:
import numpy as np
import panel as pn

pn.extension('vtk') 

data_matrix = np.zeros([75, 75, 75], dtype=np.uint8)
data_matrix[0:35, 0:35, 0:35] = 50
data_matrix[25:55, 25:55, 25:55] = 100
data_matrix[45:74, 45:74, 45:74] = 150

pn.pane.VTKVolume(data_matrix, width=800, height=600, spacing=(3,2,1), interpolation='nearest', edge_gradient=0, sampling=0)

# Show  data 

In [ ]:
import numpy as np
import panel as pn

# VTK is needed for  VTKVolumePlot 
pn.extension('vtk') 

def MyApp(endh=21, width=700, height=1000):

    data=list(ExecuteBoxQuery(db, access=access, timestep=0, endh=endh, num_refinements=1))[0]['data']
    print(f"Got data shape={data.shape} dtype={data.dtype}")

     # generate a panel for numpy data
    volume = pn.panel(
        data, 
        sizing_mode='stretch_both', 
        height=200, 
        orientation_widget =True,
        display_slices=True,
        spacing = (1, 1, 1),
        controller_expanded=False)

    # other widgets, with callbacks
    experiment  = pn.widgets.IntSlider(name='Experiment', start=0, end=1, step=1, value=0 )
    @pn.depends(experiment)
    def experiment_callback(value):
        nonlocal volume
        data=list(ExecuteBoxQuery(db, access=access, endh=endh,timestep=value,num_refinements=1))[0]['data']
        volume.object = data
        return " "    
    
    slider_i = pn.widgets.IntSlider(name='i-slice', start=0, end=data.shape[0], value = data.shape[0]//2 )
    @pn.depends(slider_i)
    def slider_i_callback(value):
        nonlocal volume
        volume.slice_i = value 
        return " "    
    
    slider_j = pn.widgets.IntSlider(name='j-slice', start=0, end=data.shape[1], value = data.shape[1]//2 )
    @pn.depends(slider_j)
    def slider_j_callback(value):
        nonlocal volume
        volume.slice_j = value 
        return " "    
    
    slider_k = pn.widgets.IntSlider(name='k-slice', start=0, end=data.shape[2], value = data.shape[2]//2 )
    @pn.depends(slider_k)
    def slider_k_callback(value):
        nonlocal volume
        volume.slice_k = value 
        return " "    
    
    show_volume = pn.widgets.Checkbox(name='Show Volume',value=True)
    @pn.depends(show_volume)
    def show_volume_callback(value):
        nonlocal volume
        volume.display_volume = value 
        return " "    
    
    show_slices = pn.widgets.Checkbox(name='Show Slices',value=True)
    @pn.depends(show_slices)
    def show_slices_calback(value):
        nonlocal volume
        volume.display_slices = value 
        return " "    

    main_layout=pn.Column(
        experiment,
        slider_i,
        slider_j,
        slider_k,
        pn.Row(
            show_volume,
            show_slices
        ),
        volume, 
        experiment_callback,
        slider_i_callback,
        slider_j_callback,
        slider_k_callback,
        show_volume_callback,
        show_slices_calback,
        width = width,
        height = height,
        css_classes=['panel-widget-box'],
        sizing_mode='stretch_width',
        width_policy='max')

    return main_layout

MyApp()

VTKVolume(ndarray, edge_gradient=0, height=600, interpolation='nearest', sampling=0, sizing_mode='fixed', slice_i=37, slice_j=37, slice_k=37, spacing=(3, 2, 1), width=800)